In [1]:
# 1) find the target kernel name and
# 2) read gpu trace file
# 3) read gpu metrics
# 4) rearrange the information and combine them in the same dataframe
# 5) save the result as csv file

In [2]:
%load_ext autoreload
%autoreload 2

import warnings
import pandas as pd
import numpy as np
import os
import sys # error msg, add the modules
import operator # sorting
from math import *
import matplotlib.pyplot as plt

sys.path.append('../')

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

import prep

In [3]:
bs_size = [128, 256, 512, 1024]

target_kern_name = 'mergeSortSharedKernel'

out_columns = None
df_kern = None
count = 0

for bs in bs_size:
    trace_file = 'trace'+str(bs)+'.csv'
    metrics_file = 'metrics'+str(bs)+'.csv'
    
    # read trace and metrics into dataframe, from generated trace file
    df_kernel_trace, df_metrics = prep.Prep_trace_metrics(trace_file, metrics_file)
    
    if count == 0:
        out_columns = prep.init_df_columns(df_metrics)
        # empty df
        df_kern = pd.DataFrame(columns=out_columns)
    
    # current kernel trace
    df_current = prep.GenCurKernInfo(df_kernel_trace, df_metrics,target_kern_name, out_columns)
    
    # add current trace
    df_kern = df_kern.append(df_current, ignore_index=True)


    count += 1


In [4]:
df_kern

,kern_name,duration_ms,gridx,gridy,gridz,blkx,blky,blkz,reg_per_thread,shared_mem,...,l2_utilization,l2_atomic_throughput,l2_atomic_transactions,sysmem_utilization,ecc_throughput,sysmem_read_utilization,sysmem_write_utilization,eligible_warps_per_cycle,flop_sp_efficiency,flop_dp_efficiency
0,mergesortsharedkernel,2.299050,32768.0,1.0,1.0,64.0,1.0,1.0,16.0,1000.0,...,Low (1),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),4.847541,0.000000%,0.000000%
1,mergesortsharedkernel,3.439707,16384.0,1.0,1.0,128.0,1.0,1.0,16.0,2000.0,...,Low (1),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),4.815699,0.000000%,0.000000%
2,mergesortsharedkernel,4.266695,8192.0,1.0,1.0,256.0,1.0,1.0,16.0,4000.0,...,Low (1),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),4.736619,0.000000%,0.000000%
3,mergesortsharedkernel,5.246929,4096.0,1.0,1.0,512.0,1.0,1.0,16.0,8000.0,...,Low (1),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),4.624627,0.000000%,0.000000%


In [5]:
df_kern.to_csv('results.csv', index=False, encoding='utf-8')